In [1]:
import os
import tqdm
import numpy as np
import pandas as pd

# API Setup

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

# Load Data

In [3]:
# Load embedding model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", embed_batch_size=32)

/Users/akashsaravanan/Library/Caches/pypoetry/virtualenvs/genai-bootcamp-4wh1UwyX-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# Load the index from disk
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.core import VectorStoreIndex

vector_store = LanceDBVectorStore(
    uri="./lancedb", table_name="pipeline_test"
)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embedding_model,
)

# Set up LLM

In [5]:
# OpenAI
from llama_index.llms.openai import OpenAI

# Set up OpenAI client - API key is handled in your .env file
openai_llm = OpenAI(model="gpt-4o", temperature=0.1, max_tokens=2048)

In [6]:
# AnyScale
from llama_index.llms.anyscale import Anyscale

# Set up AnyScale client - API key is handled in your .env file
anyscale_llm = Anyscale(model="meta-llama/Meta-Llama-3-70B-Instruct", temperature=0.1, max_tokens=2048)

# Setup RAG

In [7]:
from llama_index.core.prompts import PromptTemplate

# We put our system prompt into the template we're using to simplift things.
prompt = """You are a helpful AI assistant that answers questions after carefully reading all the provided context. You always cite sources (document titles) and also quote the relevant snippets. Information may be spread across multiple documents. If the information is not present in any of the contexts, you will say 'I don't know'.
-----
Context: 
{context_str} 
-----
Question: {query_str} 
Answer: `answer`
Source: `source document title`
Relevant Snippet: `snippet`
"""

# Convert prompt into a prompt template that llamaindex can use
prompt = PromptTemplate(template=prompt)

In [8]:
user_input = "How many points did Michael Jordan actually score in his final NBA game?"

In [9]:
# A query engine combines a retriever + a language model into a single object

# OpenAI
openai_query_engine = index.as_query_engine(llm=openai_llm, text_qa_template=prompt, similarity_top_k=3)

# Anyscale
anyscale_query_engine = index.as_query_engine(llm=anyscale_llm, text_qa_template=prompt, similarity_top_k=3)

In [10]:
# OpenAI
results = openai_query_engine.query(user_input)
for node in results.source_nodes:
    print(node.text[:100], node.score)
print("---" * 10)
print(results.response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6666018962860107
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.6664695143699646
Jordan led the league with 28.7 points per game , securing his fifth regular-season MVP award , plus 0.6528033018112183
------------------------------
**Answer:** Michael Jordan scored 15 points in his final NBA game.

**Source:** Michael Jordan

**Relevant Snippet:** "After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter... At 1:45, Jordan was intentionally fouled by the 76ers' Eric Snow, and stepped to the line to make both free throws."


In [11]:
# AnyScale
results = anyscale_query_engine.query(user_input)
for node in results.source_nodes:
    print(node.text[:100], node.score)
print("---" * 10)
print(results.response)

With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6666018962860107
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.6664695143699646
Jordan led the league with 28.7 points per game , securing his fifth regular-season MVP award , plus 0.6528033018112183
------------------------------
Answer: 13 points
Source: Michael Jordan
Relevant Snippet: "After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter and with his team trailing the Philadelphia 76ers, 75–56."


# Reranking

Reranking involves a two stage search process - a fast, less accurate search to narrow down the search space followed by a slower search that prioritizes accuracy. 

In this case we perform a search through our vector DB created earlier and use [cross-encoder/ms-marco-MiniLM-L-2-v2](https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-2-v2), an open-source (Apache 2.0) cross-encoder model. The former searches for the 10 most similar nodes while the latter narrows it down to 3.

In [12]:
from llama_index.core.postprocessor import SentenceTransformerRerank
reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-4-v2", top_n=5
)

/Users/akashsaravanan/Library/Caches/pypoetry/virtualenvs/genai-bootcamp-4wh1UwyX-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
# OpenAI
openai_rerank_query_engine = index.as_query_engine(
    llm=openai_llm,
    text_qa_template=prompt,
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

# Anyscale
anyscale_rerank_query_engine = index.as_query_engine(
    llm=anyscale_llm,
    text_qa_template=prompt,
    similarity_top_k=10,
    node_postprocessors=[reranker],
)

In [14]:
# OpenAI
results = openai_rerank_query_engine.query(user_input)
for node in results.source_nodes:
    print(node.text[:100], node.score)
print("---" * 10)
print(results.response)

Jordan led the league with 28.7 points per game , securing his fifth regular-season MVP award , plus 7.501506
Jordan also holds the top career regular season and playoff scoring averages of 30.1 and 33.4 points 7.0058327
Jordan led the league in scoring with 30.4 ppg , and won the league 's regular season and All-Star G 6.9607058
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  6.7087536
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 6.416082
------------------------------
Michael Jordan scored 15 points in his final NBA game.

Source: Michael Jordan
Relevant Snippet: "Jordan's final NBA game was on April 16, 2003 in Philadelphia. After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter and with his team trailing the Philadelphia 76ers, 75 – 56. ... At 1:45, Jordan was intentionally fouled by the 76ers' Eric Sno

In [15]:
# AnyScale
results = anyscale_rerank_query_engine.query(user_input)
for node in results.source_nodes:
    print(node.text[:100], node.score)
print("---" * 10)
print(results.response)

Jordan led the league with 28.7 points per game , securing his fifth regular-season MVP award , plus 7.501506
Jordan also holds the top career regular season and playoff scoring averages of 30.1 and 33.4 points 7.0058327
Jordan led the league in scoring with 30.4 ppg , and won the league 's regular season and All-Star G 6.9607058
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  6.7087536
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 6.416082
------------------------------
Answer: 13 points
Source: Michael Jordan
Relevant Snippet: "After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter and with his team trailing the Philadelphia 76ers, 75–56."


# Keyword / Lexical Search

Below is an example of a lexical search method - [BM25](https://en.wikipedia.org/wiki/Okapi_BM25). We create a retriever from the same chunked documents that we created for the semantic search approach from before.

In [16]:
import utils
data = utils.load_data(sample_size=100)
documents = utils.preprocess_data(data)
nodes = utils.chunk_documents(documents)

Repo card metadata block was not found. Setting CardData to empty.
100%|██████████| 100/100 [00:00<00:00, 4380.15it/s]


Parsing nodes:   0%|          | 0/100 [00:00<?, ?it/s]

Documents before chunking: 100
Documents after chunking: 837


In [17]:
from llama_index.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=3)

In [18]:
# This query relies on semantic meaning so the poor results are not surprising
bm25_results = bm25_retriever.retrieve(user_input)
for result in bm25_results:
    print(result.text[:100], result.score)

Jordan led the league in scoring with 30.4 ppg , and won the league 's regular season and All-Star G 20.286742252961535
In later years , the NBA shortened its three-point line to 22 feet ( from 23 feet , 9 inches ) , whi 19.37173182877688
He became the only player other than Wilt Chamberlain to score 3,000 points in a season , averaging  18.505265737705894


In [40]:
# OpenAI
print(openai_query_engine.synthesize(user_input, bm25_results).response)

I don't know. The provided context does not contain information about the number of points Michael Jordan scored in his final NBA game.


In [41]:
# AnyScale
print(anyscale_query_engine.synthesize(user_input, bm25_results).response)

I don't know.

The provided context does not mention Michael Jordan's final NBA game or the number of points he scored in it. The context primarily focuses on his achievements and performances during his playing career, but it does not provide information about his final game.


# Rank Fusion

[Reciprocal Rank Fusion (RRF)](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) is a commonly used rank fusion algorithm. LlamaIndex has an implementation of the same within `llama_index.core.retrievers.QueryFusionRetriever`. For illustration purposes, we implement the rank fusion algorithm below.

In [19]:
# The retriever in both query engines are the same, so we use only one
anyscale_results = anyscale_query_engine.retrieve(user_input)
for result in anyscale_results:
    print(result.text[:100], result.score)

With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6666018962860107
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.6664695143699646
Jordan led the league with 28.7 points per game , securing his fifth regular-season MVP award , plus 0.6528033018112183


In [20]:
# Ref: https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/fusion_retriever.py

def reciprocal_rank_fusion(results):
    fused_scores = {}
    text_to_node = {}
    k = 60.0 # See paper referenced at the top of this section
    # Compute Reciprocal Rank
    for nodes_with_scores in results:
        for rank, node_with_score in enumerate(
            sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)
    # Sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )
    reranked_nodes = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score
    return reranked_nodes

In [21]:
reranked = reciprocal_rank_fusion([bm25_results, anyscale_results])
for result in reranked:
    print(result.text[:100], result.score)

Jordan led the league in scoring with 30.4 ppg , and won the league 's regular season and All-Star G 0.016666666666666666
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.016666666666666666
In later years , the NBA shortened its three-point line to 22 feet ( from 23 feet , 9 inches ) , whi 0.01639344262295082
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.01639344262295082
He became the only player other than Wilt Chamberlain to score 3,000 points in a season , averaging  0.016129032258064516
Jordan led the league with 28.7 points per game , securing his fifth regular-season MVP award , plus 0.016129032258064516


In [42]:
# OpenAI
print(openai_query_engine.synthesize(user_input, reranked).response)

Michael Jordan scored 15 points in his final NBA game.

Source: Michael Jordan
Relevant Snippet: "Jordan finally rose from the bench and re-entered the game, replacing Larry Hughes with 2:35 remaining. At 1:45, Jordan was intentionally fouled by the 76ers' Eric Snow, and stepped to the line to make both free throws. After the second foul shot, the 76ers in-bounded the ball to rookie John Salmons, who in turn was intentionally fouled by Bobby Simmons one second later, stopping time so that Jordan could return to the bench."


In [43]:
# AnyScale
print(anyscale_query_engine.synthesize(user_input, reranked).response)

Answer: 13
Source: Michael Jordan
Relevant Snippet: "After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter and with his team trailing the Philadelphia 76ers, 75–56."


# Query Rewriting

In this next example, we prompt an LLM to generate several variants of a given query. We then perform retrieval for each query.

In [22]:
# Ref: https://docs.llamaindex.ai/en/latest/examples/query_transformations/query_transform_cookbook/#query-rewriting-custom
# Define a prompt to generate a number of queries similar to a given query
query_gen_str = """\
You are a helpful assistant that generates similar search queries based on a \
single input query. Generate {num_queries} search queries, one on each line, \
related to the following input query. Do not include anything else in your response.
Query: {query}
Queries:
"""
query_gen_prompt = PromptTemplate(query_gen_str)

In [44]:
def generate_queries(query: str, llm, num_queries: int = 4):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    # assumes LLM properly put each query on a newline
    queries = response.split("\n")
    return queries

In [45]:
# Anyscale
anyscale_queries = generate_queries(query=user_input, llm=anyscale_llm, num_queries=4) + [user_input]

print("\n".join(anyscale_queries))

What was the date of Michael Jordan's final NBA game?
What team did Michael Jordan play for in his final NBA game?
What was the outcome of Michael Jordan's final NBA game?
What was Michael Jordan's highest scoring performance in his NBA career?
How many points did Michael Jordan actually score in his final NBA game?


In [46]:
# Store all results but see only the top result per query
all_anyscale_results = []
for query in anyscale_queries:
    results = anyscale_query_engine.retrieve(query)
    all_anyscale_results.append(results)
    print(results[0].text[:100], results[0].score)

With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6940546035766602
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6894277334213257
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6841335296630859
Jordan also holds the top career regular season and playoff scoring averages of 30.1 and 33.4 points 0.7003832459449768
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6666018962860107


In [47]:
# OpenAI
openai_queries = generate_queries(query=user_input, llm=openai_llm, num_queries=4) + [user_input]

print("\n".join(openai_queries))

Michael Jordan final NBA game points scored
Michael Jordan last game NBA points total
Michael Jordan retirement game scoring stats
Michael Jordan final match NBA score details
How many points did Michael Jordan actually score in his final NBA game?


In [48]:
# Store all results but see only the top result per query
all_openai_results = []
for query in openai_queries:
    results = openai_query_engine.retrieve(query)
    all_openai_results.append(results)
    print(results[0].text[:100], results[0].score)

Jordan also holds the top career regular season and playoff scoring averages of 30.1 and 33.4 points 0.7494400143623352
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.7365968227386475
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.7230042219161987
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.714702308177948
With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.6666018962860107


## Rank Fusion (Again)

Let's try the rank fusion algorithm we saw previously again on the results from above.

In [49]:
# AnyScale
reranked = reciprocal_rank_fusion(all_anyscale_results)
for result in reranked:
    print(result.text[:100], result.score)

With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.06666666666666667
In March 1995 , Jordan decided to quit baseball due to the ongoing Major League Baseball strike , as 0.03278688524590164
In his 1998 autobiography For the Love of the Game , Jordan wrote that he had been preparing for ret 0.03252247488101534
Jordan also holds the top career regular season and playoff scoring averages of 30.1 and 33.4 points 0.016666666666666666
In later years , the NBA shortened its three-point line to 22 feet ( from 23 feet , 9 inches ) , whi 0.01639344262295082
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.01639344262295082
= Michael Jordan = 
 
 Michael Jeffrey Jordan ( born February 17 , 1963 ) , also known by his initia 0.016129032258064516
Jordan led the league in scoring with 30.4 ppg , and won the league 's regular season and All-Star G 0.016129032258064516
Among his numerous accomplish

In [50]:
print(reranked[0].text)

With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him throughout the NBA . In his final game at his old home court , the United Center in Chicago , Jordan received a four-minute standing ovation . The Miami Heat retired the number 23 jersey on April 11 , 2003 , even though Jordan never played for the team . At the 2003 All-Star Game , Jordan was offered a starting spot from Tracy McGrady and Allen Iverson , but refused both . In the end he accepted the spot of Vince Carter , who decided to give it up under great public pressure . 
 Jordan 's final NBA game was on April 16 , 2003 in Philadelphia . After scoring only 13 points in the game , Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter and with his team trailing the Philadelphia 76ers , 75 – 56 . Just after the start of the fourth quarter , the First Union Center crowd began chanting " We want Mike ! " . After much encouragement from coach Doug Collins , Jo

In [51]:
# OpenAI
reranked = reciprocal_rank_fusion(all_openai_results)
for result in reranked:
    print(result.text[:100], result.score)

With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him thro 0.06559139784946236
In an injury-plagued 2001 – 02 season , he led the team in scoring ( 22.9 ppg ) , assists ( 5.2 apg  0.04972677595628415
Jordan led the league with 28.7 points per game , securing his fifth regular-season MVP award , plus 0.048651507139079855
Jordan also holds the top career regular season and playoff scoring averages of 30.1 and 33.4 points 0.03279569892473118
In March 1995 , Jordan decided to quit baseball due to the ongoing Major League Baseball strike , as 0.03278688524590164
Jordan averaged a league leading 33.6 ppg on 52.6 % shooting , to go with 6.9 rpg and 6.3 apg in lea 0.01639344262295082


In [52]:
print(reranked[0].text)

With the recognition that 2002 – 03 would be Jordan 's final season , tributes were paid to him throughout the NBA . In his final game at his old home court , the United Center in Chicago , Jordan received a four-minute standing ovation . The Miami Heat retired the number 23 jersey on April 11 , 2003 , even though Jordan never played for the team . At the 2003 All-Star Game , Jordan was offered a starting spot from Tracy McGrady and Allen Iverson , but refused both . In the end he accepted the spot of Vince Carter , who decided to give it up under great public pressure . 
 Jordan 's final NBA game was on April 16 , 2003 in Philadelphia . After scoring only 13 points in the game , Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter and with his team trailing the Philadelphia 76ers , 75 – 56 . Just after the start of the fourth quarter , the First Union Center crowd began chanting " We want Mike ! " . After much encouragement from coach Doug Collins , Jo

# Query Transformation: HyDE

We examine two scenarios - one where we don't use HyDE and one where we do. 

## Without Hyde



In [32]:
user_input = "How many points did Michael Jordan actually score in his final NBA game?"

In [33]:
print(openai_query_engine.query(user_input))

Michael Jordan scored 15 points in his final NBA game.

Source: Michael Jordan
Relevant Snippet: "Jordan's final NBA game was on April 16, 2003 in Philadelphia. After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter... At 1:45, Jordan was intentionally fouled by the 76ers' Eric Snow, and stepped to the line to make both free throws."


In [34]:
print(anyscale_query_engine.query(user_input))

Answer: 13 points
Source: Michael Jordan
Relevant Snippet: "After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the third quarter and with his team trailing the Philadelphia 76ers, 75–56."


## With HyDE

In [35]:
# Query transforms perform query transformations such as expansion, HyDE, etc.
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
openai_hyde = HyDEQueryTransform(llm=openai_llm, include_original=True)
anyscale_hyde = HyDEQueryTransform(llm=anyscale_llm, include_original=True)

In [36]:
# Query engine applies a query transformation before retrieval/answer generation
from llama_index.core.query_engine import TransformQueryEngine
anyscale_hyde_query_engine = TransformQueryEngine(anyscale_query_engine, anyscale_hyde)
openai_hyde_query_engine = TransformQueryEngine(openai_query_engine, anyscale_hyde)

In [37]:
# Show the HyDE prompt
print(anyscale_hyde.get_prompts()["hyde_prompt"].template)

Please write a passage to answer the question
Try to include as many key details as possible.


{context_str}


Passage:"""



In [38]:
# Anyscale
print("Hypothetical document:")
print(anyscale_hyde.run(user_input).custom_embedding_strs[0])
print("---" * 30)
print(anyscale_hyde_query_engine.query(user_input))

Hypothetical document:
Here is a passage answering the question:

Michael Jordan's final NBA game took place on April 18, 2003, when his Washington Wizards faced off against the Philadelphia 76ers in the first round of the Eastern Conference playoffs. In a game that would ultimately be a 107-87 loss for the Wizards, Jordan, then 40 years old, played 37 minutes and scored 15 points on 6-of-15 shooting from the field, including 1-of-3 from three-point range. He also added 4 rebounds, 2 assists, and 2 steals to his stat line. Although the game marked the end of Jordan's illustrious 15-year NBA career, his 15-point performance was a testament to his enduring skill and competitive drive, even in the twilight of his playing days.
------------------------------------------------------------------------------------------
Answer: 13 points
Source: Michael Jordan
Relevant Snippet: "After scoring only 13 points in the game, Jordan went to the bench with 4 minutes and 13 seconds remaining in the t

In [39]:
# OpenAI
print("Hypothetical document:")
print("OpenAI:", openai_hyde.run(user_input).custom_embedding_strs[0])
print("---" * 30)
print(openai_hyde_query_engine.query(user_input))

Hypothetical document:
OpenAI: Michael Jordan, widely regarded as one of the greatest basketball players of all time, played his final NBA game on April 16, 2003. This game was between the Washington Wizards, the team Jordan was playing for at the time, and the Philadelphia 76ers. Despite the Wizards' loss with a final score of 107-87, Jordan's performance was a memorable moment in basketball history. In his last appearance on the NBA court, Michael Jordan scored 15 points. This game marked the end of an illustrious career that included six NBA championships, five MVP awards, and numerous other accolades. Jordan's final points came from a free throw, and he received a standing ovation from the crowd, which included fans, fellow players, and even opponents, all paying tribute to his legendary career.
------------------------------------------------------------------------------------------
Michael Jordan scored 15 points in his final NBA game.

Source: Michael Jordan
Relevant Snippet: "